In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

# Load project data

In [ ]:
from mapswipe.data_access import get_project_data, read_scoped_projects_list, CACHE_PATH, CACHE_SIZE, get_user_metrics
df_projects = read_scoped_projects_list()
print(len(df_projects))
validate_projects = list(df_projects[df_projects["project_type"] == 2]["project_id"])

In [ ]:
df_projects[["project_type", "status"]].value_counts().sort_index()

In [ ]:
df_user_metrics = get_user_metrics(True)

In [ ]:
df_user_metrics.head()

In [ ]:
df_user_metrics["involved_project_count"].describe()

In [ ]:
(pd.Series(logistic.cdf(df_user_metrics["involved_project_count"], loc=15, scale=3) * 4) + 1.0).describe()

In [ ]:
import numpy as np
from scipy.stats import logistic
import matplotlib.pyplot as plt

plt.subplots(figsize=(7, 7))

plt.scatter(df_user_metrics["involved_project_count"], pd.Series(logistic.cdf(df_user_metrics["involved_project_count"], loc=15, scale=3) * 4) + 1.0, s=4)
plt.show()

In [ ]:
plt.subplots(figsize=(7, 7))
plt.scatter(df_user_metrics["involved_project_count"], df_user_metrics["user_weight"], s=4)

## Inspect project summaries

In [ ]:
df_projects.head()

In [ ]:
df_projects[df_projects["project_type"] == 2].describe()

In [ ]:
(df_projects["number_of_results"] / df_projects["area_sqkm"]).describe()

In [ ]:
df_projects[df_projects["project_type"] == 2].sort_values("area_sqkm").tail(5)

In [ ]:
df_projects[df_projects["project_id"] == "-MRL3frZWPOCR94ehFnp"]

# Load an individual project
This uses the disk cache so make sure `mapswipe.data_access.CACHE_PATH` is set correctly

In [ ]:
data = get_project_data("-NQz3e_OZzUvi22pL0Ul")

In [ ]:
data["agg"].tail()

In [ ]:
data["agg"].describe()

## Seed the cache

In [ ]:
for project_id in validate_projects:
    try:
        get_project_data(project_id)
    except:
        print(f"Error getting {project_id}")
        raise

# Spatial Correlation

In [ ]:
from functools import partial
from splot.esda import plot_moran
from mapswipe.project_stats import calc_moran_for_knn, safe_calc_moran, calc_moran_for_dist, calc_moran_for_dist_debug, calc_moran_for_dist_weighted

## Debugging tools

In [ ]:
project_id = "-NQz3e_OZzUvi22pL0Ul"

data = get_project_data(project_id)

In [ ]:
data["agg"].head()

In [ ]:
data["agg"].describe()

In [ ]:
agree_k_func = partial(calc_moran_for_knn, col_name="incorrect_score", k_vals=(1, 2, 3, 5, 10, 15))
df_moran_agree = safe_calc_moran(project_id, agree_k_func)

Scree plot for a single project

In [ ]:
correct_dist_func = partial(calc_moran_for_dist, col_name="incorrect_score", dist_vals=(5.0, 10.0, 20.0, 25.0, 50.0, 75.0, 100.0, 200.0, 300.0))
df_moran_dist_correct = safe_calc_moran(project_id, correct_dist_func)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(df_moran_dist_correct["dist"], df_moran_dist_correct["moran_i"], "ro-", linewidth=2)

## agreement / KNN

In [ ]:
import pandas as pd
import multiprocessing as mp
from functools import partial

moran_tables = []

#moran_func = partial(safe_calc_moran, col_name="agreement", k=(1, 2, 3, 5, 10, 15))
agree_k_func = partial(calc_moran_for_knn, col_name="agreement", k_vals=(1, 2, 3, 5, 10, 15))

with mp.Pool(processes=8) as pool:
    #map_args = [(p, "agreement", (1, 2, 3, 5, 10, 15)) for p in validate_projects]
    map_args = [(p, agree_k_func) for p in validate_projects]
    moran_tables = pool.starmap(safe_calc_moran, map_args)
    moran_tables = [t for t in moran_tables if t is not None]

df_moran_agreement = pd.concat(moran_tables)

In [ ]:
df_moran_agreement.boxplot(column="moran_i", by="k")

## incorrect_score / KNN

In [ ]:
import pandas as pd
import multiprocessing as mp
from functools import partial

moran_tables = []

correct_k_func = partial(calc_moran_for_knn, col_name="incorrect_score", k_vals=(1, 2, 3, 5, 10, 15))

with mp.Pool(processes=8) as pool:
    map_args = [(p, correct_k_func) for p in validate_projects]
    moran_tables = pool.starmap(safe_calc_moran, map_args)
    moran_tables = [t for t in moran_tables if t is not None]

df_moran_incorrect_score = pd.concat(moran_tables)

In [ ]:
df_moran_incorrect_score.boxplot(column="moran_i", by="k")

## incorrect_score / DistanceBand
Calculating moran by distance is very memory-intensive so keep the parallelism low. This takes a long time for 80+ projects

In [ ]:
import pandas as pd
import multiprocessing as mp
from functools import partial

moran_tables = []

correct_dist_func = partial(calc_moran_for_dist, col_name="incorrect_score", dist_vals=(5.0, 10.0, 20.0, 25.0, 50.0, 75.0, 100.0, 200.0, 300.0))

with mp.Pool(processes=1) as pool:
    map_args = [(p, correct_dist_func) for p in validate_projects]
    moran_tables = pool.starmap(safe_calc_moran, map_args)
    moran_tables = [t for t in moran_tables if t is not None]

df_moran_correct_dist = pd.concat(moran_tables)

In [ ]:
df_moran_correct_dist.boxplot(column="moran_i", by="dist").set_ylim(top=1.0)

In [ ]:
df_moran_correct_dist[df_moran_correct_dist["moran_i"] >= 2.0].head()

In [ ]:
df_moran_correct_dist[df_moran_correct_dist["project_id"] == "-NC5JLsuWA0V6nbNfbRo"]

In [ ]:
import pandas as pd
import multiprocessing as mp
from functools import partial

moran_tables = []

correct_dist_w_func = partial(calc_moran_for_dist_weighted, col_name="incorrect_score", dist_vals=(5.0, 10.0, 20.0, 25.0, 50.0, 75.0, 100.0, 200.0, 300.0))

with mp.Pool(processes=1) as pool:
    map_args = [(p, correct_dist_w_func) for p in validate_projects]
    moran_tables = pool.starmap(safe_calc_moran, map_args)
    moran_tables = [t for t in moran_tables if t is not None]

df_moran_correct_dist_w = pd.concat(moran_tables)

In [ ]:
df_moran_correct_dist_w.boxplot(column="moran_i", by="dist").set_ylim(top=1.0)

# Population Statistics

# Visual Inspection

In [ ]:
import folium
import branca.colormap as cm

def create_task_map(gdf, center_pt=None):
    
    geojson_data = gdf.drop('lastEdit', axis=1).to_json()

    if center_pt is None:
        center_pt = gdf.to_crs(gdf.estimate_utm_crs()).dissolve().centroid.to_crs(4326)
    map = folium.Map(location=[center_pt.y, center_pt.x], zoom_start=8)
    map._repr_html_ = lambda: map._parent._repr_html_(
    include_link=False, width='75%', height='400px'
    )

    colormap = cm.linear.YlOrRd_09.scale(gdf["1_share"].min(), gdf["1_share"].max())

    def style_function(feature):
        return {
            'fillColor': colormap(feature['properties']['1_share']),
            'color': 'black',
            'weight': 0.5,
            'fillOpacity': 0.8
        }

    folium.GeoJson(
        geojson_data,
        style_function=style_function,
        name="geojson"
    ).add_to(map)

    colormap.add_to(map)

    return map

## Investigate a incorrect_score / DistanceBand outlier

In [ ]:
correct_dist_data = get_project_data("-NC5JLsuWA0V6nbNfbRo")
correct_dist_moran = calc_moran_for_dist_debug(correct_dist_data["agg"], "incorrect_score", 25.0)
plot_moran(correct_dist_moran)

In [ ]:
create_task_map(correct_dist_data["agg"])

In [ ]:
df_moran_incorrect_score[df_moran_incorrect_score["moran_i"] >= 0.8].head(20)

In [ ]:
correct_dist_data["agg"].describe()

In [ ]:
import matplotlib.pyplot as plt
#plt.scatter(correct_dist_data["agg"]["incorrect_score"], correct_dist_data["agg"]["nearby_building_count"])
#correct_dist_data["agg"].plot(x="incorrect_score", y="nearby_building_count", type="dot")
import numpy as np
print(np.corrcoef(correct_dist_data["agg"]["incorrect_score"], correct_dist_data["agg"]["nearby_building_count"]))

from scipy.stats import pearsonr, spearmanr
print(pearsonr(correct_dist_data["agg"]["incorrect_score"], correct_dist_data["agg"]["nearby_building_count"]))
print(spearmanr(correct_dist_data["agg"]["incorrect_score"], correct_dist_data["agg"]["nearby_building_count"]))

## Investigate a weird outlier
This project isn't a typical project, it's based on some AI-generated data:
https://download.geoservice.dlr.de/WSF2019/

In [ ]:
weird_data = get_project_data("-MRL3frZWPOCR94ehFnp")

In [ ]:
weird_data["agg"].estimate_utm_crs()

In [ ]:
weird_data["agg"].dissolve().to_crs(weird_data["agg"].estimate_utm_crs()).centroid.to_crs(4326)

In [ ]:
import shapely
# Taken from the project summary df
weird_center = shapely.Point(-5.100499450000003, 7.735347999999997)

In [ ]:
create_task_map(weird_data["agg"].head(1000), weird_center)